
# 1. Import Dependencies¶


In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy



# 2. Types of Spaces


In [4]:
Discrete(3).sample()

2

In [5]:
Box(0,1,shape=(3,3)).sample()
Box(0,255,shape=(3,3), dtype=int).sample()

array([[100,  70, 196],
       [206, 112, 204],
       [ 57,  12, 117]])

In [9]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

(0, array([46.174618], dtype=float32))

In [10]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([15.95309], dtype=float32))])

In [11]:
MultiBinary(4).sample()

array([1, 0, 0, 1], dtype=int8)

In [12]:
MultiDiscrete([5,2,2]).sample()

array([1, 0, 0])

with MultiDiscrete exemple :
- first value : between 0 and 4
- second value : between 0 and 1
- third value : same as previous


# 3. Building an Environment¶


Our goal is to build an agent ot give us the best shower possible
- Randomly temperature
- 37 and 39 degrees (the magic ratio)


In [13]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # = Box(low = 0, high = 100, shape(1,)  )
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):

        # Apply action
        # action - 1
        # 0 -1 = -1 temperature -> decrease temperature (action = 0)
        # 1 -1 = 0 -> no change temperature (action = 1)
        # 2 -1 = 1 temperature -> increase temperature (action = 2)
        
        # the new temperature
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: # it's the magic ratio. Cf markdown above
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state


In [14]:
env = ShowerEnv()

/home/uruk380/anaconda3/envs/Reinforcement_Learning_YT_course/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [15]:
env.observation_space.sample()

array([82.28524], dtype=float32)

In [16]:
env.action_space.sample()

2

#  4. Test Environment


In [17]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()


Episode:1 Score:10
Episode:2 Score:-52
Episode:3 Score:-46
Episode:4 Score:-44
Episode:5 Score:-30



# 5. Train Model


In [18]:
log_path = os.path.join('Training_Custom_Env', 'Logs')

In [19]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [22]:
model.learn(total_timesteps=2000)

Logging to Training_Custom_Env/Logs/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | 55.8     |
| time/              |          |
|    fps             | 1680     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 56.3        |
| time/                   |             |
|    fps                  | 1245        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010633253 |
|    clip_fraction        | 0.141       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.552      |
|    explained_variance   | 7.


# 6. Save Model¶


In [23]:
shower_path = os.path.join('Training_Custom_Env','Saved Model','Shoxer_Model_PPO')

In [24]:
model.save(shower_path)

In [25]:
del model

In [26]:
model = PPO.load(shower_path,env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [29]:
evaluate_policy(model, env,n_eval_episodes=10,render=False)

/home/uruk380/anaconda3/envs/Reinforcement_Learning_YT_course/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(59.2, 0.9797958971132712)